# Lab 5

Advanced topics with APIs: Scaling

## Prompt caching and Batch APIs

In [1]:
from dotenv import load_dotenv
from litellm import completion
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="pydantic.*")
load_dotenv(override=True)

True

In [2]:
with open("hamlet.txt", "r", encoding="utf-8") as f:
    hamlet = f.read()

loc = hamlet.find("Speak, man")
print(hamlet[loc:loc+100])

Speak, man.
  Laer. Where is my father?
  King. Dead.
  Queen. But not by him!
  King. Let him deman


In [3]:
question = "In Hamlet, when Laertes asks 'Where is my father?' what is the reply?"

In [4]:
messages = [{"role": "user", "content": question}]
response = completion(model="gpt-4.1-nano", messages=messages)
reply = response.choices[0].message.content
print(reply)

In Act IV, Scene 5 of *Hamlet*, when Laertes asks, "Where is my father?" the reply is that Ophelia has gone mad and is wandering, and the King (Claudius) responds by saying that she is crazy and, in her madness, has lost her wits. Specifically, if you refer to the lines, the response is:

**"Ophelia, no, my lord,  
Poisoned we are, born with a sneer,  
When they talk of deeds, [mighty deeds]  
In our madness, we think it best to die,  
And forget ourselves."**

However, the most direct reply to Laertes' question "Where is my father?" is that Laertes' father, Polonius, has been slain by Hamlet, and he is mourning him. The players and other characters inform Laertes that Polonius is dead, and this knowledge propels his grief and anger.

**In summary:**  
When Laertes asks, "Where is my father?" the commonplace reply within the context of the scene is that Polonius has been killed by Hamlet, and Laertes learns of his father’s death, prompting his grief and desire for revenge.


In [5]:
print(f"Input tokens: {response.usage.prompt_tokens}")
print(f"Output tokens: {response.usage.completion_tokens}")
print(f"Total cost: {response._hidden_params["response_cost"]*100:.4f} cents")

Input tokens: 25
Output tokens: 250
Total cost: 0.0103 cents


In [6]:
system = f"""
You are a helpful assistant that can answer questions about the text of Hamlet.
For context, here is the entire text of Hamlet:
{hamlet}
"""

In [7]:
messages = [{"role":"system", "content": system}, {"role": "user", "content": question}]
response = completion(model="gpt-4.1-nano", messages=messages)
reply = response.choices[0].message.content
print(reply)

In Hamlet, when Laertes asks "Where is my father?" the reply is "Dead."


In [8]:
print(f"Input tokens: {response.usage.prompt_tokens}")
print(f"Output tokens: {response.usage.completion_tokens}")
print(f"Cached tokens: {response.usage.prompt_tokens_details.cached_tokens}")
print(f"Total cost: {response._hidden_params["response_cost"]*100:.4f} cents")

Input tokens: 49706
Output tokens: 19
Cached tokens: 0
Total cost: 0.4978 cents


In [9]:
#AGAIN!! cached

response = completion(model="gpt-4.1-nano", messages=messages)
reply = response.choices[0].message.content
print(reply)

In Hamlet, when Laertes asks "Where is my father?" the reply is "Dead."


In [10]:
print(f"Input tokens: {response.usage.prompt_tokens}")
print(f"Output tokens: {response.usage.completion_tokens}")
print(f"Cached tokens: {response.usage.prompt_tokens_details.cached_tokens}")
print(f"Total cost: {response._hidden_params["response_cost"]*100:.4f} cents")

Input tokens: 49706
Output tokens: 19
Cached tokens: 49536
Total cost: 0.1263 cents


## Prompt Caching with OpenAI

For OpenAI:

https://platform.openai.com/docs/guides/prompt-caching

> Cache hits are only possible for exact prefix matches within a prompt. To realize caching benefits, place static content like instructions and examples at the beginning of your prompt, and put variable content, such as user-specific information, at the end. This also applies to images and tools, which must be identical between requests.


Cached input is 4X cheaper

https://openai.com/api/pricing/

## Prompt Caching with Anthropic

https://docs.anthropic.com/en/docs/build-with-claude/prompt-caching

You have to tell Claude what you are caching

You pay 25% MORE to "prime" the cache

Then you pay 10X less to reuse from the cache with inputs.

https://www.anthropic.com/pricing#api

## Gemini supports both 'implicit' and 'explicit' prompt caching

https://ai.google.dev/gemini-api/docs/caching?lang=python